In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


In [2]:
n_rows = 300000
df = pd.read_csv("train.gz", nrows=n_rows, compression="infer")

In [3]:
for c in df.columns:
    df[c]=df[c].apply(str)
    le=preprocessing.LabelEncoder().fit(df[c])
    df[c] =le.transform(df[c])
    pd.to_numeric(df[c]).astype(np.float)

In [4]:
X = df.drop(['click', 'id', 'hour', 'device_id', 'device_ip'], axis=1).values
Y = df['click'].values


In [5]:
n_train = int(n_rows * 0.9)
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]

In [6]:
enc = OneHotEncoder(handle_unknown='ignore')
X_train_enc = enc.fit_transform(X_train)
X_test_enc = enc.transform(X_test)

In [7]:
n_features = int(X_train_enc.toarray().shape[1])
learning_rate = 0.001
n_iter = 50

In [8]:
# Input and Target placeholders
x = tf.placeholder(tf.float32, shape=[None, n_features])
y = tf.placeholder(tf.float32, shape=[None])

In [9]:
# Build the logistic regression model
W = tf.Variable(tf.zeros([n_features, 1]))
b = tf.Variable(tf.zeros([1]))

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
logits = tf.add(tf.matmul(x, W), b)[:, 0]
pred = tf.nn.sigmoid(logits)

In [11]:
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))
auc = tf.metrics.auc(tf.cast(y, tf.int64), pred)[1]

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [13]:
# Initialize the variables
init_vars = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [14]:
batch_size = 1000

In [15]:
indices = list(range(n_train))

In [16]:
def gen_batch(indices):
    np.random.shuffle(indices)
    for batch_i in range(int(n_train / batch_size)):
        batch_index = indices[batch_i*batch_size: (batch_i+1)*batch_size]
        yield X_train_enc[batch_index], Y_train[batch_index]

In [17]:
sess = tf.Session()

sess.run(init_vars)

In [18]:
for i in range(1, n_iter+1):
    avg_cost = 0.
    for X_batch, Y_batch in gen_batch(indices):
        _, c = sess.run([optimizer, cost], feed_dict={x: X_batch.toarray(), y: Y_batch})
        avg_cost += c / int(n_train / batch_size)
    print('Iteration %i, training loss: %f' % (i, avg_cost))

Iteration 1, training loss: 0.464202
Iteration 2, training loss: 0.414743
Iteration 3, training loss: 0.409080
Iteration 4, training loss: 0.405974
Iteration 5, training loss: 0.403799
Iteration 6, training loss: 0.402161
Iteration 7, training loss: 0.400820
Iteration 8, training loss: 0.399711
Iteration 9, training loss: 0.398795
Iteration 10, training loss: 0.397979
Iteration 11, training loss: 0.397265
Iteration 12, training loss: 0.396620
Iteration 13, training loss: 0.396049
Iteration 14, training loss: 0.395520
Iteration 15, training loss: 0.395071
Iteration 16, training loss: 0.394638
Iteration 17, training loss: 0.394227
Iteration 18, training loss: 0.393846
Iteration 19, training loss: 0.393504
Iteration 20, training loss: 0.393150
Iteration 21, training loss: 0.392868
Iteration 22, training loss: 0.392578
Iteration 23, training loss: 0.392318
Iteration 24, training loss: 0.392053
Iteration 25, training loss: 0.391800
Iteration 26, training loss: 0.391579
Iteration 27, trainin

In [19]:
auc_test = sess.run(auc, feed_dict={x: X_test_enc.toarray(), y: Y_test})
print("AUC of ROC on testing set:", auc_test)

AUC of ROC on testing set: 0.76679647
